## Additional Experiments 

## Quantization 

In [1]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import torch.quantization
import time

Flash attention 2 is not installed


In [2]:
prompts = [
    "Hello world.",
    "A futuristic AI assistant responding in a clear, robotic yet friendly manner",
    "A dramatic voice-over for an action-packed movie trailer that builds suspense",
    "A casual and friendly conversation starter that feels natural and engaging",
    "An enchanting introduction to a bedtime story that sparks imagination",
    "A detailed weather update that provides temperature, wind speed, and overall forecast",
    "A robotic voice delivering a monotone yet precise system status update",
    "A powerful and energetic motivational speech that inspires action and confidence"
]

descriptions = [
    "A female speaker delivers a warm and welcoming message with a slightly expressive and friendly tone. The speech has a moderate pace and a natural intonation, making it feel inviting. The recording is clear, with minimal background noise.",
    "A robotic AI voice speaks in a neutral yet slightly friendly manner. The speech is steady, with a consistent pitch and minimal variation in tone. The audio is crisp, resembling a synthesized assistant's response.",
    "A deep male voice narrates with a dramatic and intense tone, building suspense with pauses and rising intensity. The speech is slow-paced, emphasizing key moments. The recording is cinematic, with a slight reverberation for a grand effect.",
    "A young adult male speaker talks in a casual and relaxed tone. The speech is natural, with slight variations in pitch and pauses that mimic real-life conversations. The recording is high quality, making it feel like a personal chat.",
    "A soft-spoken female speaker introduces a bedtime story with a soothing and melodic tone. The pace is slow and gentle, making it easy to follow. The recording has a slight warmth, resembling a close-up microphone capture.",
    "A professional male voice delivers a clear and informative weather update. The tone is neutral but engaging, with a moderate pace. The articulation is precise, and the recording is high quality, with no distortions.",
    "A synthetic robotic voice speaks in a completely monotone and even-paced manner. The pitch remains constant, with no emotional inflection. The recording is crisp and clean, resembling an automated system response.",
    "A dynamic and energetic male speaker delivers an inspiring speech with a strong, enthusiastic tone. The pace is varied, with emphasis on key words to motivate the listener. The recording is sharp and immersive, with no background noise."
]

In [3]:
device =  "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")

/home/system4/anaconda3/envs/tf-gpu-211/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "rela

In [4]:
input_ids = tokenizer(descriptions, return_tensors="pt", padding="max_length", max_length=70).to(device)
prompt_input_ids = tokenizer(prompts, return_tensors="pt", padding="max_length", max_length=70).to(device)

In [5]:
quantized_model = torch.quantization.quantize_dynamic(
    model, 
    {torch.nn.Linear}, 
    dtype=torch.qint8
)

In [6]:
start_time = time.time()
generation = quantized_model.generate(input_ids=input_ids.input_ids,attention_mask=input_ids.attention_mask, 
                                        prompt_input_ids=prompt_input_ids.input_ids,prompt_attention_mask=prompt_input_ids.attention_mask,)
end_time = time.time()
print(f"Quantized model generation time: {end_time - start_time:.2f} seconds")

`prompt_attention_mask` is specified but `attention_mask` is not. A full `attention_mask` will be created. Make sure this is the intended behaviour.


Quantized model generation time: 240.78 seconds


In [7]:
start_time = time.time()
generation = model.generate(input_ids=input_ids.input_ids,attention_mask=input_ids.attention_mask, 
                                prompt_input_ids=prompt_input_ids.input_ids,prompt_attention_mask=prompt_input_ids.attention_mask)
end_time = time.time()
print(f"Normal model generation time: {end_time - start_time:.2f} seconds")

Normal model generation time: 203.39 seconds


## Using the wrapper to trace the model for TorchScript conversion.

In [168]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import time

In [241]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1.1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1.1")

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.49.0",
  "use_cache": true,
  "vocab_size": 32128
}

Config of the audio_encoder: <class 'transformers.models.dac.modelin

In [182]:
prompt = "Hey, how are you doing today?"
description = "A female speaker with a slightly low-pitched voice delivers her words quite expressively."
input_ids = tokenizer(description, return_tensors="pt").to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
class ParlerTTSWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, input_ids, prompt_input_ids):

        return self.model.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
        
        )

In [ ]:
model.eval()
wrapped_model = ParlerTTSWrapper(model).to(device)

In [243]:
with torch.no_grad():
    test_output = wrapped_model(input_ids, prompt_input_ids)
    print("Test run successful!")

Test run successful!


In [ ]:
with torch.no_grad():
    traced_model = torch.jit.trace(
        wrapped_model, 
        (input_ids, prompt_input_ids)
    )

## Converting to ONNX

In [1]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import os

Flash attention 2 is not installed


In [20]:
class ParlerTTSWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, input_ids, prompt_input_ids):
        return self.model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            prompt_input_ids=prompt_input_ids,
            prompt_attention_mask=torch.ones_like(prompt_input_ids),
            max_length=500,  
            pad_token_id=self.model.config.pad_token_id,
            do_sample=False, 
            num_beams=1, 
        )

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")

/home/system4/anaconda3/envs/tf-gpu-211/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "rela

In [21]:
wrapped_model = ParlerTTSWrapper(model)
wrapped_model.eval()

ParlerTTSWrapper(
  (model): ParlerTTSForConditionalGeneration(
    (text_encoder): T5EncoderModel(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=768, out_features=768, bias=False)
                  (k): Linear(in_features=768, out_features=768, bias=False)
                  (v): Linear(in_features=768, out_features=768, bias=False)
                  (o): Linear(in_features=768, out_features=768, bias=False)
                  (relative_attention_bias): Embedding(32, 12)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseGatedActDense(
                  (wi_0):

In [12]:
prompt = "Hey, how are you doing today?"
description = "A female speaker with a slightly low-pitched voice delivers her words quite expressively."
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

In [22]:
with torch.no_grad():
    test_output = wrapped_model(input_ids, prompt_input_ids)
    print("Test run successful!")

Test run successful!


In [23]:
dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'description_sequence_length'},
    'prompt_input_ids': {0: 'batch_size', 1: 'prompt_sequence_length'},
    'output': {0: 'batch_size', 1: 'audio_length'}
}

In [24]:
output_path = "parler_tts_mini.onnx"

In [26]:
with torch.no_grad():
    torch.onnx.export(
        wrapped_model,                  
        (input_ids, prompt_input_ids),  
        output_path,                    
        export_params=True,             
        opset_version=15,               
        do_constant_folding=True,       
        input_names=['input_ids', 'prompt_input_ids'],  
        output_names=['output'],        
        dynamic_axes=dynamic_axes,      
        verbose=True
    )

ValueError: `min_length` has to be a non-negative integer, but is 51